# GPT-2 124M Training on Shakespeare Dataset

This notebook trains a decoder-only 124M parameter GPT-2 model on Shakespeare text.

**Target**: Train until loss < 0.1

**Hardware Recommendation**: GPU (T4 or better)


## Setup and Dependencies


In [ ]:
# Install required packages
%pip install torch tiktoken transformers matplotlib


In [ ]:
import os
import math
import time
import inspect
from dataclasses import dataclass
import torch
import torch.nn as nn
from torch.nn import functional as F
import tiktoken
import json
from datetime import datetime


## Upload Input Data

Upload your `input.txt` file using the file upload button or use the code below:


In [ ]:
from google.colab import files

# Upload input.txt file
print("Please upload your input.txt file:")
uploaded = files.upload()

# Verify the file exists
if 'input.txt' in uploaded:
    print("✓ input.txt uploaded successfully!")
    with open('input.txt', 'r') as f:
        text = f.read()
        print(f"File size: {len(text):,} characters")
else:
    print("❌ Please upload input.txt")


## Model Architecture


In [ ]:
class CausalSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = 1
        # regularization
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.register_buffer(
            "bias", 
            torch.tril(torch.ones(config.block_size, config.block_size))
            .view(1, 1, config.block_size, config.block_size)
        )

    def forward(self, x):
        B, T, C = x.size()  # batch size, sequence length, embedding dimensionality (n_embd)
        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)  # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)  # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)  # (B, nh, T, hs)

        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        y = att @ v  # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)

        y = y.transpose(1, 2).contiguous().view(B, T, C)  # re-assemble all head outputs side by side
        y = self.c_proj(y)
        return y


class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd)
        self.gelu = nn.GELU(approximate='tanh')
        self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = 1

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x


class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x


In [ ]:
@dataclass
class GPTConfig:
    block_size: int = 1024  # max sequence length
    vocab_size: int = 50257  # number of tokens: 50,000 BPE merges + 256 bytes tokens + 1 <|endoftext|>
    n_layer: int = 12  # number of layers
    n_head: int = 12  # number of heads
    n_embd: int = 768  # embedding dimension


class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte=nn.Embedding(config.vocab_size, config.n_embd),
            wpe=nn.Embedding(config.block_size, config.n_embd),
            h=nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f=nn.LayerNorm(config.n_embd),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        # weight sharing
        self.transformer.wte.weight = self.lm_head.weight

        # weight initialization
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            std = 0.02
            if hasattr(module, 'NANOGPT_SCALE_INIT'):
                std *= (2 * self.config.n_layer) ** -0.5
            torch.nn.init.normal_(module.weight, mean=0.0, std=std)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.size()
        assert T <= self.config.block_size, f"Cannot forward sequence of length {T}, block size is only {self.config.block_size}"
        
        pos = torch.arange(0, T, dtype=torch.long, device=idx.device)
        pos_emb = self.transformer.wpe(pos)
        tok_emb = self.transformer.wte(idx)
        x = tok_emb + pos_emb
        
        for block in self.transformer.h:
            x = block(x)
        
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x)
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss

    def get_num_params(self):
        """Return the number of parameters in the model"""
        return sum(p.numel() for p in self.parameters())


# Print model info
config = GPTConfig()
temp_model = GPT(config)
print(f"Model Configuration:")
print(f"  Layers: {config.n_layer}")
print(f"  Heads: {config.n_head}")
print(f"  Embedding dim: {config.n_embd}")
print(f"  Total parameters: {temp_model.get_num_params():,}")
del temp_model


## Data Loader


In [ ]:
class DataLoaderLite:
    def __init__(self, B, T, split='train'):
        self.B = B
        self.T = T

        # Load tokens from disk
        with open('input.txt', 'r') as f:
            text = f.read()
        enc = tiktoken.get_encoding('gpt2')
        tokens = enc.encode(text)
        self.tokens = torch.tensor(tokens)
        print(f'Loaded {len(self.tokens):,} tokens')
        print(f'1 epoch = {len(self.tokens) // (B * T):,} batches')

        # State
        self.current_position = 0

    def next_batch(self):
        B, T = self.B, self.T
        buf = self.tokens[self.current_position: self.current_position + B * T + 1]
        x = (buf[:-1]).view(B, T)  # inputs
        y = (buf[1:]).view(B, T)   # targets
        
        # Advance position
        self.current_position += B * T
        
        # Reset if out of bounds
        if self.current_position + (B * T + 1) > len(self.tokens):
            self.current_position = 0
        
        return x, y


## Training Configuration


In [ ]:
# Device setup
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
print(f"Using device: {device}")

# Training hyperparameters
BATCH_SIZE = 16  # Increased from 4 for better training
SEQUENCE_LENGTH = 128  # Increased from 32
LEARNING_RATE = 3e-4
MAX_STEPS = 10000  # Will stop early if target loss is reached
TARGET_LOSS = 0.0999999  # Target loss threshold
EVAL_INTERVAL = 50  # Evaluate every N steps
SAVE_INTERVAL = 500  # Save checkpoint every N steps

# Create directories
os.makedirs('checkpoints', exist_ok=True)
os.makedirs('logs', exist_ok=True)

print(f"\nTraining Configuration:")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Sequence length: {SEQUENCE_LENGTH}")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  Max steps: {MAX_STEPS}")
print(f"  Target loss: {TARGET_LOSS}")
print(f"  Device: {device}")


## Initialize Model and Optimizer


In [ ]:
# Set random seeds for reproducibility
torch.manual_seed(1337)
if torch.cuda.is_available():
    torch.cuda.manual_seed(1337)

# Initialize model
model = GPT(GPTConfig())
model.to(device)
print(f"Model initialized with {model.get_num_params():,} parameters")

# Initialize data loader
train_loader = DataLoaderLite(B=BATCH_SIZE, T=SEQUENCE_LENGTH)

# Initialize optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

print("\n✓ Model and optimizer ready!")


## Training Loop with Comprehensive Logging


In [ ]:
# Training logs
training_logs = []
start_time = time.time()
best_loss = float('inf')

print("\n" + "="*80)
print("STARTING TRAINING")
print("="*80 + "\n")

for step in range(MAX_STEPS):
    # Training step
    step_start_time = time.time()
    
    x, y = train_loader.next_batch()
    x, y = x.to(device), y.to(device)
    
    optimizer.zero_grad()
    logits, loss = model(x, y)
    loss.backward()
    optimizer.step()
    
    step_time = time.time() - step_start_time
    
    # Log training metrics
    log_entry = {
        'step': step,
        'loss': loss.item(),
        'time': step_time,
        'timestamp': datetime.now().isoformat()
    }
    training_logs.append(log_entry)
    
    # Print progress
    if step % EVAL_INTERVAL == 0 or step == MAX_STEPS - 1:
        elapsed = time.time() - start_time
        tokens_processed = step * BATCH_SIZE * SEQUENCE_LENGTH
        tokens_per_sec = tokens_processed / elapsed if elapsed > 0 else 0
        
        print(f"Step {step:5d} | Loss: {loss.item():.6f} | "
              f"Time: {step_time:.3f}s | "
              f"Tokens/s: {tokens_per_sec:.0f} | "
              f"Elapsed: {elapsed:.0f}s")
    
    # Save checkpoint (periodic - includes optimizer for resuming training)
    if step > 0 and step % SAVE_INTERVAL == 0:
        checkpoint_path = f'checkpoints/model_step_{step}.pt'
        torch.save({
            'step': step,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),  # Needed to resume training
            'loss': loss.item(),
            'config': config.__dict__,
        }, checkpoint_path)
        print(f"  💾 Checkpoint saved: {checkpoint_path}")
    
    # Check if target loss reached
    if loss.item() < TARGET_LOSS:
        print(f"\n{'='*80}")
        print(f"🎉 TARGET LOSS REACHED! Loss: {loss.item():.6f} < {TARGET_LOSS}")
        print(f"{'='*80}\n")
        
        # Save final model (deployment version - NO optimizer state)
        # This creates a smaller file (~500MB vs ~1.4GB)
        final_checkpoint_path = 'checkpoints/model_final.pt'
        torch.save({
            'step': step,
            'model_state_dict': model.state_dict(),
            'loss': loss.item(),
            'config': config.__dict__,
        }, final_checkpoint_path)
        print(f"✓ Final model saved: {final_checkpoint_path} (~500MB, deployment-ready)")
        
        # Also save full version with optimizer (for potential fine-tuning)
        full_checkpoint_path = 'checkpoints/model_final_full.pt'
        torch.save({
            'step': step,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss.item(),
            'config': config.__dict__,
        }, full_checkpoint_path)
        print(f"✓ Full checkpoint saved: {full_checkpoint_path} (~1.4GB, for fine-tuning)")
        break
    
    # Update best loss (save deployment version)
    if loss.item() < best_loss:
        best_loss = loss.item()
        
        # Save deployment version (without optimizer - smaller file)
        best_checkpoint_path = 'checkpoints/model_best.pt'
        torch.save({
            'step': step,
            'model_state_dict': model.state_dict(),
            'loss': loss.item(),
            'config': config.__dict__,
        }, best_checkpoint_path)
        print(f"  ⭐ Best model updated: {best_checkpoint_path} (loss: {best_loss:.6f})")

# Save training logs
log_file = f'logs/training_log_{datetime.now().strftime("%Y%m%d_%H%M%S")}.json'
with open(log_file, 'w') as f:
    json.dump(training_logs, f, indent=2)

total_time = time.time() - start_time
print(f"\n{'='*80}")
print("TRAINING COMPLETE")
print(f"{'='*80}")
print(f"Total time: {total_time:.2f}s ({total_time/60:.2f} minutes)")
print(f"Final loss: {loss.item():.6f}")
print(f"Best loss: {best_loss:.6f}")
print(f"Total steps: {step + 1}")
print(f"Training log saved: {log_file}")
print(f"{'='*80}\n")


## 📦 Checkpoint Files Explained

### File Sizes & Purpose

The training saves different types of checkpoints:

| Checkpoint | Size | Contains | Purpose |
|------------|------|----------|---------|
| **model_final.pt** | ~500 MB | Model weights only | ✅ **Use this for deployment** (Hugging Face, production) |
| **model_final_full.pt** | ~1.4 GB | Model + Optimizer | For fine-tuning or resuming training |
| **model_step_X.pt** | ~1.4 GB | Model + Optimizer | Periodic checkpoints during training |
| **model_best.pt** | ~500 MB | Model weights only | Best loss checkpoint (deployment-ready) |

### Why the Size Difference?

- **Model weights**: 124M parameters × 4 bytes = ~500 MB
- **Optimizer state** (Adam): 124M × 2 × 4 bytes = ~900 MB (momentum + variance)
- **Total with optimizer**: ~1.4 GB

### Which File to Use?

✅ **For Hugging Face deployment**: Use `model_final.pt` (~500 MB)
- Faster upload
- Lower storage costs  
- Perfect for inference

📚 **For future training**: Keep `model_final_full.pt` (~1.4 GB)
- Can resume training
- Can fine-tune on new data
- Includes optimizer state


In [ ]:
import matplotlib.pyplot as plt

# Plot training loss
steps = [log['step'] for log in training_logs]
losses = [log['loss'] for log in training_logs]

plt.figure(figsize=(12, 6))
plt.plot(steps, losses, linewidth=0.5, alpha=0.6, label='Training Loss')
plt.axhline(y=TARGET_LOSS, color='r', linestyle='--', label=f'Target Loss ({TARGET_LOSS})')
plt.xlabel('Step')
plt.ylabel('Loss')
plt.title('Training Loss Over Time')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('logs/training_loss.png', dpi=150, bbox_inches='tight')
plt.show()

print("Loss plot saved: logs/training_loss.png")


## Test Text Generation


In [ ]:
# Generate sample text
model.eval()
enc = tiktoken.get_encoding('gpt2')

num_return_sequences = 3
max_length = 100
prompt = "First Citizen:"

print("\nGenerating sample text...\n")
print("="*80)

tokens = enc.encode(prompt)
tokens = torch.tensor(tokens, dtype=torch.long).unsqueeze(0).repeat(num_return_sequences, 1)
x = tokens.to(device)

torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

while x.size(1) < max_length:
    with torch.no_grad():
        logits, _ = model(x)
        logits = logits[:, -1, :]
        probs = F.softmax(logits, dim=-1)
        topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
        ix = torch.multinomial(topk_probs, 1)
        xcol = torch.gather(topk_indices, -1, ix)
        x = torch.cat((x, xcol), dim=1)

for i in range(num_return_sequences):
    tokens = x[i, :max_length].tolist()
    decoded = enc.decode(tokens)
    print(f"Sample {i+1}:")
    print(decoded)
    print("\n" + "-"*80 + "\n")

model.train()


## Download Model Weights

Download the trained model weights to use in Hugging Face Spaces


In [ ]:
from google.colab import files
import shutil

# Create a zip file with all checkpoints and logs
shutil.make_archive('trained_model', 'zip', '.', 'checkpoints')
shutil.make_archive('training_logs', 'zip', '.', 'logs')

print("Downloading model weights and logs...")
files.download('trained_model.zip')
files.download('training_logs.zip')
print("\n✓ Downloads complete!")
print("\nYou can now use these weights in the Hugging Face Spaces app.")
